# Production

**Ситуация**: Вы работает data scientist в крупном продуктовом российском ритейлере. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи   
**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

**Ожидание:**
- Отправляем e-mail с топ-10 товарами, отсортированными по вероятности

**Реальность:**
- Чего хочет менеджер от рекомендательной системы? (рост показателя X на Y% за Z недель)
- По-хорошему надо бы предварительно посчитать потенциальный эффект от рекоммендательной системы (Оценки эффектов у менеджера и у вас могут сильно не совпадать: как правило, вы знаете про данные больше)
- А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они?
- Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе?
- Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?)
- Какие товары должны быть в e-mail? Есть ли какие-то ограничения (только акции и т п)?
- Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки
- Cколько мы хотим зарабатывать с одного привлеченного юзера?
---
**Попытаться ответить на вопросы/выдвинуть гипотезы:**

    1.А точно нужно сортировать по вероятности?
    2.Какую метрику использовать?
    3.Сколько раз в неделю отправляем рассылку?
    4.В какое время отправляем рассылку?
    5.Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?
    6.Нужно ли, чтобы в одной рассылке были разные товары? Как определить, что товары разные? Как добиться того, чтобы они были разными?


**В итоге договорились, что:**
- Хотим повысить выручку минимум на 6% за 4 месяца. Будем повышать за счет роста Retention минимум на  3% и среднего чека минимум на 3%
- Топ-5 товаров, а не топ-10 (В e-mail 10 выглядят не красиво, в push и на чек больше 5 не влезает)
- Рассылаем в e-mail (5% клиентов) и push-уведомлении (20% клиентов), печатаем на чеке (все оффлайн клиенты)
- **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)
- **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?) 
- **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)

Вопросов стало еще больше. Поэтому сначала делаем **MVP** (Minimum viable product) на e-mail. Показываем его менеджеру, измеряем метрики на юзерах. По фидбеку и метрикам делаем улучшения MVP и раскатываем его на push-уведомления и чеки

*Data Science проект* - итеративный процесс!

# ДЗ

1) **Попытаться ответить на вопросы/выдвинуть гипотезы:**


1.**А точно нужно сортировать по вероятности?**

Вероятнее всего сортировку предподчтительнее осуществлять по потенциальной выручке с товара нежели по вероятности. 

2.**Какую метрику использовать?**

Money precision - она учитывает то, что мы рекомендуем именно релевантные товары, а также стоимость каждого рекомендованного товара.

3.**Сколько раз в неделю отправляем рассылку?**

Оптимально будет два раза в неделю:

- В середине недели (среда) для поддержания интереса к нашим товарам и к спотанным покупкам, что будет способствовать увеличению среднего чека.

- В пятницу непосредвенно перед выходными, когда будет осуществляться большая закупка продуктов "на неделю".

4.**В какое время отправляем рассылку?**

- Среда - в районе 14-15 ч незадолго до конца раб. дня.
- Пятница - в вечернее время, когда люди начинают формировать планы на выходные.

5.**Будем отправлять одному юзеру много раз наши рекомендации. Как добиться того, чтобы они хоть немного отличались?**

Дать некоторым категориям товаров увеличенный вес в соответствии со днем недели рассылки. К примеру формировать в середине неделирекомендации по хоз.товарам и в конце недели по алкоголю и бакалее.

2) **Доделать прошлые домашние задания**

# Links

BM25

https://en.wikipedia.org/wiki/Okapi_BM25#:~:text=BM25%20is%20a%20bag%2Dof,slightly%20different%20components%20and%20parameters.


Matrix factorization (ALS, SVD)
https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/

Выполнено.

3) **Прочитать статьи BM25/MatrixFactorization**

Прочитано.

## Практика:

4) Поэкспериментировать с ALS (grid-search) ********

In [8]:
import itertools as it

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.model_selection import GridSearchCV
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
    
from metrics import precision_at_k, recall_at_k
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
data = pd.read_csv('retail_train.csv')

In [3]:
data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
            inplace=True)

In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [5]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [7]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [9]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

In [10]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float)

In [10]:

sparse_user_item = csr_matrix(user_item_matrix)

In [11]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

ALS

In [22]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [12]:
def score_als_precision(model, user_ids, actual_recs, N=5):
    recs = user_ids.apply(lambda x: get_recommendations(x, model=model, N=N))
    result = pd.DataFrame(actual_recs)
    result = result.rename(columns={'user_id': 'actual'})
    result['recs'] = recs
    score = result.apply(lambda row: precision_at_k(row['recs'], row['actual']), axis=1).mean()
    
    return score

In [13]:
rec_results = data_test.groupby('user_id')['item_id'].unique().reset_index()
rec_results.columns=['user_id', 'actual']
rec_results.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [14]:
parameters = {'factors': [32, 64, 128],
              'regularization': [0.0001, 0.001, 0.01],
              'iterations': [10, 15, 20]}

In [58]:
%%time

scores = []
for factors, reg, iters in it.product(*parameters.values()):
    model = AlternatingLeastSquares(factors=factors, 
                                    regularization=reg,
                                    iterations=iters, 
                                    calculate_training_loss=True, 
                                    num_threads=6)

    model.fit(csr_matrix(user_item_matrix).T,  # На вход item-user matrix
              show_progress=True)
    
    score = {'factors': factors,
             'regularization': reg,
             'iterations': iters,
             'score': score_als_precision(model, rec_results['user_id'], rec_results['actual'], N=5)}
    scores.append(score)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Wall time: 17min 4s


In [61]:
pd.DataFrame(scores, index=range(len(scores))).sort_values('score', ascending=False)

,factors,regularization,iterations,score
7,32,0.0100,15,0.170813
12,64,0.0010,10,0.169050
17,64,0.0100,20,0.169050
15,64,0.0100,10,0.168560
3,32,0.0010,10,0.168168
2,32,0.0001,20,0.167385
6,32,0.0100,10,0.166601
8,32,0.0100,20,0.166503
0,32,0.0001,10,0.166308
4,32,0.0010,15,0.166112


Параметры оптимальной модели:
1.factors = 32

2.regularization = 0.01
3.iterations = 15
